# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Producer** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team Members**:

- Alberto Renteria Camacho
- Lorena Ruelas Gaytán
- Ximena Isaac Horta
- Yael Alejandro Rodríguez Barreto

In [1]:
from kafka import KafkaProducer
import threading
import json
import random
from time import sleep

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = '66817d0dbacb:9093'
TOPIC_PREFIX = 'tweet-'

Create the worker function for multithread

In [2]:
def worker(user_id: int, topic: str):

    num_messages = random.randint(300, 500)

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            sleep(1.5)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrupted.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer closed.")

In [4]:
if __name__ == "__main__":
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Espera a que todos los hilos terminen
    for t in threads:
        t.join()

    print("Finished.")


[Thread-2] Iniciando, topic=tweet-2, mensajes=365
[Thread-1] Iniciando, topic=tweet-1, mensajes=310
[Thread-4] Iniciando, topic=tweet-4, mensajes=409
[Thread-3] Iniciando, topic=tweet-3, mensajes=358
[Thread-2] Sent to tweet-2: 9ca221a9-0f63-4b79-946d-58dae2fb2aca
[Thread-4] Sent to tweet-4: 6e728e89-65dd-4e1d-8875-d94b4de35ed2
[Thread-1] Sent to tweet-1: 49fbcaf1-d5cc-49e0-926e-eca2b91147bf
[Thread-3] Sent to tweet-3: 98653804-6f7a-4f4c-ba1a-ada0fb9394f1
[Thread-2] Sent to tweet-2: cc060745-b8a8-467b-812a-9db1d15812b6
[Thread-1] Sent to tweet-1: 295e9a53-47d3-4458-ad9b-807867890b8d
[Thread-4] Sent to tweet-4: aef1d7df-fc47-4ec7-85d5-0e886434419c
[Thread-3] Sent to tweet-3: 49992d22-a77d-401c-8e0d-66aa218a6b85
[Thread-2] Sent to tweet-2: 107c3139-d969-492e-97f8-7d6c51bd5c1b
[Thread-1] Sent to tweet-1: 5abf5039-c5df-4103-a465-38c409ab546e
[Thread-4] Sent to tweet-4: 48f6b7c8-1eee-4660-b80c-5567e50aff1b
[Thread-3] Sent to tweet-3: 3cd371d0-e853-4e1b-8857-a238cd016736
[Thread-2] Sent to t